This is inspired from a Notebook prepared by Louise called "Synthetized_beam_animation.Rmd"

In [ ]:
import glob
import os 

import numpy as np
import healpy as hp

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

%matplotlib inline
#%matplotlib notebook

from matplotlib import rc
rc('figure',figsize=(16,10))
rc('font',size=12)
rc('text',usetex=False)

from qubicpack.utilities import Qubic_DataDir
import qubic 
import qubic.sb_fitting as sbfit

from qubicpack.pixel_translation import make_id_focalplane, tes2index
from qubic.selfcal_lib import index2TESandASIC

######################################################################
# Dictionary
d = qubic.qubicdict.qubicDict()
d.read_from_file(os.environ['QUBIC_DICT']+'pipeline_demo.dict')
d['config'] = 'TD'
# Instrument and scene
q = qubic.QubicInstrument(d)
s = qubic.QubicScene(d)
print(q.detector.index)

# Detector centers
detpositions = q.detector.center
print(detpositions.shape)
print(detpositions[0:3,:])

# Detector exactly at the center of the fp
detcenterposition = np.array([0,0,-0.3])

In [ ]:
rc('figure',figsize=(10,10))
scatter(detpositions[:, 0], detpositions[:, 1], marker='s', s=400, alpha=0.3)
plot(detcenterposition[0], detcenterposition[1],'ro')
xlim(-0.055, 0.005)
ylim(-0.055, 0.005)

for k in range(248):
    text(detpositions[k, 0]-0.001, detpositions[k, 1]-0.0005, k, fontsize=10)

In [ ]:
# Synthetic beams on the sky for all TES
sb = q.get_synthbeam(s, idet=None, external_A=None, hwp_position=0)
print(sb.shape)

# Let's also take a detector at the center of the FP
sbcenter = q.get_synthbeam(s, idet=None, external_A=None, hwp_position=0, detpos=detcenterposition)

Now let's get the positions and amplitudes of all peaks

In [ ]:
# Coordinates of the peaks (spherical coordinates in radian) for each TES
kmax = 3
npeaks = (2 * kmax + 1)**2
delta_horn = q.horn.spacing
angle_horn = q.horn.angle
nu = d['filter_nu']
position_TES = q.detector.center
theta, phi = q._peak_angles_kmax(kmax, q.horn.spacing, q.horn.angle, d['filter_nu'], detpositions)
vals = np.array(q.primary_beam(theta, phi), dtype=float, copy=False)

### Now we want the same but for a virtual detector at the center of the FP - for some reason the codde crashes
### when one asks for a single TES position...  So we just do all of them placed at the center and keep only
### the first one...
detpositions_center = detpositions.copy()
detpositions_center[:,0] = 0
detpositions_center[:,1] = 0
alltheta, allphi = q._peak_angles_kmax(kmax, q.horn.spacing, q.horn.angle, d['filter_nu'], detpositions_center)
theta_0 = alltheta[0,:]
phi_0 = allphi[0,:]
vals_0 = np.array(q.primary_beam(theta_0, phi_0), dtype=float, copy=False)


In [ ]:
### Let's plot the synthesized beam and the peak positions for the virtuald etector at the center
### We see tnat they are orddered by amplitude
rc('figure',figsize=(16,5))
hp.gnomview(sbcenter, min=0, max=100000, rot=(180, 90), reso=30, title=None, cmap='viridis', sub=(1,3,1))
for p in range(npeaks):
    if p == npeaks//2:
        color = 'w'
    else:
        color = 'r'
    th = theta_0[p]
    ph = phi_0[p]
    hp.visufunc.projscatter(th, ph, color=color, marker='+')
    hp.visufunc.projtext(th+np.radians(1), ph+np.radians(1), str(p), color=color, fontsize=12)

subplot(1,3,2)
scatter(detpositions[:, 0], detpositions[:, 1], marker='s', s=40, alpha=0.3)
plot(detcenterposition[0], detcenterposition[1],'ro')

subplot(1,3,3)
plot(vals_0,'bo-')
xlabel('Peak #')
ylabel('Peak Amplitude')
#yscale('log')

In [ ]:
rc('figure',figsize=(16,5))
TES = np.random.randint(248)*0+39
hp.gnomview(sb[TES], min=0, max=100000, rot=(180, 90), reso=30, title='TES {}'.format(TES), cmap='viridis', sub=(1,3,1))
for p in range(npeaks):
    if p == npeaks//2:
        color = 'w'
    else:
        color = 'r'
    th = theta[TES, p]
    ph = phi[TES, p]
    hp.visufunc.projscatter(th, ph, color=color, marker='+')
    hp.visufunc.projtext(th, ph, str(p), color=color, fontsize=12)

subplot(1,3,2)
scatter(detpositions[:, 0], detpositions[:, 1], marker='s', s=40, alpha=0.3)
plot(detpositions[TES,0], detpositions[TES,1],'ro')
title('TES {}'.format(TES))

subplot(1,3,3)
plot(vals[TES,:],'bo-')
xlabel('Peak #')
ylabel('Peak Amplitude')
title('TES {}'.format(TES))


The peak with number npeaks//2 is always at the center. So let's plot the theta and phi of this "central" peak for all TES.

In [ ]:
hp.gnomview(sb[0]*0, min=0, max=100000, rot=(180, 90), reso=10, title='TES {}'.format(TES), cmap='viridis', sub=(1,3,1))
for TES in range(248):
    th = theta[TES, npeaks//2]
    ph = phi[TES, npeaks//2]
    hp.visufunc.projscatter(th, ph, color='r', marker='.')

hp.visufunc.projscatter(theta_0[npeaks//2], phi_0[npeaks//2], color='w', marker='.')

So with this numbering and the position of peak npeaks//2 we should be able to have a unique identifier for peaks and follow them as a function of detector and as a function of frequency and so on...